In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import os
import sklearn
from sklearn import svm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score,roc_auc_score 
from sklearn.metrics import precision_recall_fscore_support as score
from imblearn.ensemble import BalancedRandomForestClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.utils.multiclass import unique_labels
from imblearn.over_sampling import SVMSMOTE
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#The path for importing the dataset needs to be changed to where your winequality-white.csv file is locateddf = pd.read_csv('/Users/camillaandersson/Documents/Høst 2020 - NTNU/Maskinlæring/Project/winequality-white.csv', delimiter=";")
#Data visualization
df.head()

In [ ]:
# Check for missing values
print(df.isna().sum())

### Divide quality into bad, good, and average

https://www.kaggle.com/muammerhuseyinoglu/prediction-of-wine-quality

In [ ]:
#Classify the quality
quality = df["quality"].values
category = []
for num in quality:
    if num<5:
        category.append(0) #Bad
    elif num>6:
        category.append(2) #Good
    else:
        category.append(1) #Average

#Create new data
category = pd.DataFrame(data=category, columns=["category"])
data = pd.concat([df,category],axis=1)
data.drop(columns="quality",axis=1,inplace=True)

data.head()

In [ ]:
# Number of rows and columns
print("Rows, columns: " + str(data.shape))

In [ ]:
category = data['category'].value_counts()
category

In [ ]:
#Histogram
plt.figure(figsize=(10,6))
sns.countplot(data["category"],palette="muted")
plt.title('Distribution of bad, average and good quality wine', fontsize = 20)
plt.xticks(np.arange(3), ['Bad', 'Average', 'Good'])
data["category"].value_counts()

In [ ]:
#Heatmap
sns.heatmap(data.corr())
plt.show()

In [ ]:
#https://medium.com/datadriveninvestor/regression-from-scratch-wine-quality-prediction-d61195cb91c8
correlations = data.corr()['category'].sort_values(ascending=False)
print("\033[1m"+ "Correlations:"+"\033[0m")
print(correlations)

### Normalize, encode and split into train and test set

In [ ]:
X = data.drop('category', axis=1)
y = data['category']

# Encode target
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

# Standarize X so that X_train and X_test is standarized
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Train and Test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 10)

In [ ]:
print("X_train: " + str(X_train.shape))
print("y_train: " + str(y_train.shape))
print("X_test: " + str(X_test.shape))
print("y_test: " + str(y_test.shape))

# Functions

### Method for confusion matrix

https://stackoverflow.com/questions/58799204/confusion-martix-showing-the-values-with-cutting-off

In [ ]:
class_names = np.array(['0','1', '2'])

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.xlim(-0.5, len(np.unique(y))-0.5) # ADD THIS LINE
    plt.ylim(len(np.unique(y))-0.5, -0.5) # ADD THIS LINE
    return ax

np.set_printoptions(precision=2)

# KNN

## Hyperparameter Tuning

https://medium.com/datadriveninvestor/k-nearest-neighbors-in-python-hyperparameters-tuning-716734bc557f

### GridSearch Standard KNN

In [ ]:
#List Hyperparameters that we want to tune.
n_neighbors = list(range(1,60))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_base = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_base, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(X,y)
#Print The value of best Hyperparameters
print('Standard KNN')
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

### GridSearch Weighted KNN

In [ ]:
#List Hyperparameters that we want to tune.
n_neighbors = list(range(1,60))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_base_2 = KNeighborsClassifier(weights = 'distance')
#Use GridSearch
clf = GridSearchCV(knn_base_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(X,y)
#Print The value of best Hyperparameters
print('Weighted KNN')
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

## Standard KNN

https://www.kaggle.com/muammerhuseyinoglu/prediction-of-wine-quality

In [ ]:
#define model
knn_standard = KNeighborsClassifier(n_neighbors=32, p=1) 
#fit model
knn_standard.fit(X_train,y_train)
#predict model
pred_knn_standard=knn_standard.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_knn_standard_prob = knn_standard.predict_proba(X_test)

#https://www.kaggle.com/muammerhuseyinoglu/prediction-of-wine-quality
conclusion = pd.DataFrame({'model': ["Standard KNN"],
                           'accuracy': [accuracy_score(y_test,pred_knn_standard)], 
                           'F1-score': [f1_score(y_test,pred_knn_standard,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_knn_standard_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, pred_knn_standard, classes=class_names,
                      title='Confusion matrix - Standard KNN')

## Weighted KNN

In [ ]:
#define model
knn_weighted = KNeighborsClassifier(weights = 'distance',n_neighbors=48, p=1)
#fit model
knn_weighted.fit(X_train,y_train)
#predict model
pred_knn_weighted=knn_weighted.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_knn_weighted_prob = knn_weighted.predict_proba(X_test)

#https://www.kaggle.com/muammerhuseyinoglu/prediction-of-wine-quality
conclusion = pd.DataFrame({'model': ["Weighted KNN"],
                           'accuracy': [accuracy_score(y_test,pred_knn_weighted)], 
                           'F1-score': [f1_score(y_test,pred_knn_weighted,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_knn_weighted_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, pred_knn_weighted, classes=class_names,
                      title='Confusion matrix - Weighted KNN')

## KNN with SVM-SMOTE

In [ ]:
#Oversampling the data with RandomOverSampler
smote = SVMSMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
#define model
knn_smote = KNeighborsClassifier(n_neighbors=48, p=1)
#fit model
knn_smote.fit(X_train_smote,y_train_smote)
#predict model
pred_knn_smote=knn_smote.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_knn_smote_prob = knn_smote.predict_proba(X_test)

#https://www.kaggle.com/muammerhuseyinoglu/prediction-of-wine-quality
conclusion = pd.DataFrame({'model': ["KNN with SVM-SMOTE"],
                           'accuracy': [accuracy_score(y_test,pred_knn_smote)], 
                           'F1-score': [f1_score(y_test,pred_knn_smote,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_knn_smote_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, pred_knn_smote, classes=class_names,
                      title='Confusion matrix - KNN with SMOTE')

# Summary KNN

In [ ]:
summary = pd.DataFrame({'model': ["Standard KNN", "Weighted KNN", "KNN with SMOTE"], 
                           'accuracy': [100*accuracy_score(y_test,pred_knn_standard), 100*accuracy_score(y_test,pred_knn_weighted), 100*accuracy_score(y_test,pred_knn_smote)], 
                           'F1-score': [100*f1_score(y_test,pred_knn_standard,average='macro'), 100*f1_score(y_test,pred_knn_weighted,average='macro'), 100*f1_score(y_test,pred_knn_smote,average='macro')],
                          'ROC AUC': [100*roc_auc_score(y_test, pred_knn_standard_prob, multi_class = 'ovo'), 100*roc_auc_score(y_test, pred_knn_weighted_prob, multi_class = 'ovo'), 100*roc_auc_score(y_test, pred_knn_smote_prob, multi_class = 'ovo')]})

summary

# SVM

### Hyperparameter Tuning

In [ ]:
#Grid search to find the best parameters
parameters = {'kernel':('rbf', 'linear'), 'C':[0.1, 1.0, 10.0, 100.0], 'gamma':[0.5, 1, 10]}

SVC= svm.SVC()
gscv = GridSearchCV(SVC, param_grid=parameters, cv=2)
gscv.fit(X_train, y_train)

print(gscv.best_params_)

## SVM with no weighting and oversampling

In [ ]:
#SVM model 1: No weighting, no oversampling
SVC = svm.SVC(kernel='rbf', gamma= 1, C=10, probability=True)
SVC.fit(X_train, y_train)
y_pred = SVC.predict(X_test)

In [ ]:
#Results for SVM model 1
y_pred_prob = SVC.predict_proba(X_test)

conclusion = pd.DataFrame({'model': ["Standard SVM"],
                           'accuracy': [accuracy_score(y_test,y_pred)], 
                           'F1-score': [f1_score(y_test,y_pred,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, y_pred_prob, multi_class = 'ovo', average ='macro')]})
conclusion

In [ ]:
#Confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix - Standard SVM')

## SVM with balanced weights

In [ ]:
#Grid search to find the best parameters
parameters = {'kernel':('rbf', 'linear'), 'C':[0.1, 1.0, 10.0, 100.0], 'gamma':[0.5, 1, 10]}

SVC= svm.SVC(class_weight = 'balanced')
gscv = GridSearchCV(SVC, param_grid=parameters, cv=2)
gscv.fit(X_train, y_train)

print(gscv.best_params_)

In [ ]:
#SVM model 2: With balanced weights
SVC_w = svm.SVC(kernel='rbf', C=100, gamma=1, class_weight = 'balanced', probability=True)
SVC_w.fit(X_train, y_train)
y_pred_w = SVC_w.predict(X_test)

In [ ]:
#Results for SVM model 1
y_pred_prob_w = SVC_w.predict_proba(X_test)
conclusion = pd.DataFrame({'model': ["weighted SVM"],
                           'accuracy': [accuracy_score(y_test,y_pred_w)], 
                           'F1-score': [f1_score(y_test,y_pred_w,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, y_pred_prob_w, multi_class = 'ovo', average = 'macro')]})
conclusion

In [ ]:
#Confusion matrix
plot_confusion_matrix(y_test, y_pred_w, classes=class_names,
                      title='Confusion matrix - Weighted SVM')

## SVM with oversampling

In [ ]:
#Oversampling the data with SMOTE
smote = SVMSMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
#New grid search with oversampled data
parameters = {'kernel':('rbf', 'linear'), 'C':[0.1, 1.0, 10.0, 100.0], 'gamma':[0.5, 1, 10]}

SVC= svm.SVC()
gscv = GridSearchCV(SVC, param_grid=parameters, cv=4)
gscv.fit(X_train_smote, y_train_smote)

print(gscv.best_params_)

In [ ]:
#SVM model 3: SVM trained with oversampled data
SVC_smote = svm.SVC(kernel='rbf', C=100, gamma=1, probability=True)
SVC_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = SVC_smote.predict(X_test)

In [ ]:
#Results for SVM model 3:
y_pred_prob_smote = SVC_smote.predict_proba(X_test)
conclusion = pd.DataFrame({'model': ["SVM with SMOTE"],
                           'accuracy': [accuracy_score(y_test,y_pred_smote)], 
                           'F1-score': [f1_score(y_test,y_pred_smote,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, y_pred_prob_smote, multi_class = 'ovo', average = 'macro')]})
conclusion

In [ ]:
#Confusion matrix
plot_confusion_matrix(y_test, y_pred_smote, classes=class_names,
                      title='Confusion matrix - SVM with SMOTE')

# Summary SVM

In [ ]:
#Total results
summary = pd.DataFrame({'model': ["Standard SVM", "Weighted SVM", "SVM with SMOTE"], 
                           'accuracy': [100*accuracy_score(y_test,y_pred), 100*accuracy_score(y_test,y_pred_w), 100*accuracy_score(y_test,y_pred_smote)], 
                           'F1-score': [100*f1_score(y_test,y_pred,average='macro'), 100*f1_score(y_test,y_pred_w,average='macro'), 100*f1_score(y_test,y_pred_smote,average='macro')],
                          'ROC AUC': [100*roc_auc_score(y_test, y_pred_prob, multi_class = 'ovo', average = 'macro'), 100*roc_auc_score(y_test, y_pred_prob_w, multi_class = 'ovo', average = 'macro'), 100*roc_auc_score(y_test, y_pred_prob_smote, multi_class = 'ovo', average = 'macro')]})

summary

# Random Forest

### Hyperparameter Tuning

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
#Investigate default parameters
rfc = RandomForestClassifier() 
print('Parameters currently in use:\n')
pprint(rfc.get_params())

#### Find best hyperparameters using randomized search with cross validation

First define parameter values

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rfc_base = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rfc_base, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

## Standard Random Forest

In [ ]:
#define model
rfc_standard = RandomForestClassifier(n_estimators=670, min_samples_split = 2, min_samples_leaf=1, 
                                      max_features = 'auto', max_depth = 100, bootstrap = True)
#fit model
rfc_standard.fit(X_train, y_train)
#predict model
pred_rfc_standard = rfc_standard.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_rfc_standard_prob = rfc_standard.predict_proba(X_test)

#https://www.kaggle.com/muammerhuseyinoglu/prediction-of-wine-quality
conclusion = pd.DataFrame({'model': ["Random Forest"],
                           'accuracy': [accuracy_score(y_test,pred_rfc_standard)], 
                           'F1-score': [f1_score(y_test,pred_rfc_standard,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_rfc_standard_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, pred_rfc_standard, classes=class_names,
                      title='Confusion matrix - Standard RF')

## Weighted Random Forest

Find best set of hyperparameters for weighted RF

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rfc_base_weighted = RandomForestClassifier(class_weight='balanced')
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rfc_base_weighted, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
rfc_weighted = RandomForestClassifier(n_estimators=670, min_samples_split = 2, min_samples_leaf=1, 
                                      max_features = 'auto', max_depth = 100, bootstrap = True,
                                      class_weight='balanced')
rfc_weighted = rfc_weighted.fit(X_train, y_train)
pred_rfc_weighted = rfc_weighted.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_rfc_weighted_prob = rfc_weighted.predict_proba(X_test)

conclusion = pd.DataFrame({'model': ["Random Forest"],
                           'accuracy': [accuracy_score(y_test,pred_rfc_weighted)], 
                           'F1-score': [f1_score(y_test,pred_rfc_weighted,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_rfc_weighted_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
#Confusion matrix 
plot_confusion_matrix(y_test, pred_rfc_weighted, classes=class_names,
                      title='Confusion matrix - Weighted RF')

### Random Forest With Bootstrap Class Weighting

In [ ]:
rfc_BCW = RandomForestClassifier(n_estimators=670, min_samples_split = 2, min_samples_leaf=1, 
                                      max_features = 'auto', max_depth = 100, bootstrap = True,
                                      class_weight='balanced_subsample')
rfc_BCW.fit(X_train, y_train)
pred_rfc_BCW = rfc_BCW.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_rfc_BCW_prob = rfc_BCW.predict_proba(X_test)

conclusion = pd.DataFrame({'model': ["Random Forest"],
                           'accuracy': [accuracy_score(y_test,pred_rfc_BCW)], 
                           'F1-score': [f1_score(y_test,pred_rfc_BCW,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_rfc_BCW_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
#Confusion matrix 
plot_confusion_matrix(y_test, pred_rfc_BCW, classes=class_names,
                      title='Confusion matrix - Bootstrap Class Weighted RF')

## Random Forest using oversampling

### SVMSMOTE

In [ ]:
#Oversampling the data with RandomOverSampler
svmsmote = SVMSMOTE()
X_train_svmsmote, y_train_svmsmote = svmsmote.fit_resample(X_train, y_train)

In [ ]:
#define model
rfc_svmsmote = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rfc_random = RandomizedSearchCV(estimator = rfc_svmsmote, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rfc_random.fit(X_train_svmsmote, y_train_svmsmote)
rfc_random.best_params_

In [ ]:
#Define model with correct parameters
rfc_svmsmote = RandomForestClassifier(n_estimators=450, min_samples_split = 2, min_samples_leaf=1, 
                                      max_features = 'auto', max_depth = 50, bootstrap = True)

# Random forest trained with oversampled data
rfc_svmsmote.fit(X_train_svmsmote, y_train_svmsmote)
pred_rfc_svmsmote = rfc_smote.predict(X_test)

In [ ]:
pred_rfc_svmsmote_prob = rfc_svmsmote.predict_proba(X_test)

conclusion = pd.DataFrame({'model': ["Random Forest"],
                           'accuracy': [accuracy_score(y_test,pred_rfc_smote)], 
                           'F1-score': [f1_score(y_test,pred_rfc_smote,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_rfc_svmsmote_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
#Confusion matrix 
plot_confusion_matrix(y_test, pred_rfc_smote, classes=class_names,
                      title='Confusion matrix - RF with SVMSMOTE')

### SMOTE

In [ ]:
#Oversampling the data with RandomOverSampler
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
#define model
rfc_smote = RandomForestClassifier()

#Oversampling the data with RandomOverSampler
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rfc_random = RandomizedSearchCV(estimator = rfc_smote, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rfc_random.fit(X_train_smote, y_train_smote)
rfc_random.best_params_

In [ ]:
#Define model with correct parameters
rfc_smote = RandomForestClassifier(n_estimators=450, min_samples_split = 2, min_samples_leaf=1, 
                                      max_features = 'auto', max_depth = 50, bootstrap = True)

# Random forest trained with oversampled data
rfc_smote.fit(X_train_smote, y_train_smote)
pred_rfc_smote = rfc_smote.predict(X_test)

In [ ]:
#calculate probability estimates for parameter y_score in roc_auc_score:
pred_rfc_smote_prob = rfc_smote.predict_proba(X_test)

conclusion = pd.DataFrame({'model': ["Random Forest"],
                           'accuracy': [accuracy_score(y_test,pred_rfc_smote)], 
                           'F1-score': [f1_score(y_test,pred_rfc_smote,average='macro')],
                          'ROC AUC': [roc_auc_score(y_test, pred_rfc_smote_prob, multi_class = 'ovo')]})
conclusion

In [ ]:
#Confusion matrix 
plot_confusion_matrix(y_test, pred_rfc_smote, classes=class_names,
                      title='Confusion matrix - RF with SMOTE')

# Summary Random Forest

In [ ]:
summary = pd.DataFrame({'model': ["Standard Random Forest", "Weighted Random Forest", "Random Forest With Bootstrap Class Weighting", "Random Forest with SMOTE"], 
                           'accuracy': [100*accuracy_score(y_test,pred_rfc_standard), 100*accuracy_score(y_test,pred_rfc_weighted), 100*accuracy_score(y_test,pred_rfc_BCW), 100*accuracy_score(y_test,pred_rfc_smote)], 
                           'F1-score': [100*f1_score(y_test,pred_rfc_standard,average='macro'), 100*f1_score(y_test,pred_rfc_weighted,average='macro'), 100*f1_score(y_test,pred_rfc_BCW,average='macro'), 100*f1_score(y_test,pred_rfc_smote,average='macro')],
                          'ROC AUC': [100*roc_auc_score(y_test, pred_rfc_standard_prob, average='macro', multi_class = 'ovo'), 100*roc_auc_score(y_test, pred_rfc_weighted_prob, average='macro', multi_class = 'ovo'), 100*roc_auc_score(y_test, pred_rfc_BCW_prob, average='macro', multi_class = 'ovo'), 100*roc_auc_score(y_test, pred_rfc_smote_prob, average='macro', multi_class = 'ovo')]})

summary